# Binance Test Net API Research

In [14]:
async def close_websockets(loop):
    print("Closing all websockets...")
    for ws in open_websockets.copy():
        try:
            open_websockets.remove(ws)
            await ws.close()
        except Exception as e:
            print(f"Error closing websocket: {e}")
        else:
            open_websockets.remove(ws)
    print("Done")

In [20]:
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
loop.run_forever()
loop.call_soon_threadsafe(loop.create_task, close_websockets(loop))
loop.stop()

RuntimeError: Cannot run the event loop while another loop is running

In [21]:
loop.stop()

In [ ]:
### Testnet URLs
# https://testnet.binance.vision/api
# wss://testnet.binance.vision/ws-api/v3
# wss://testnet.binance.vision/ws
# wss://testnet.binance.vision/stream

# Binance Spot Test Network: https://testnet.binance.vision/
# API Key: odwF9bVsSsxjZnckgbSu3NfUgGqqJ2sow4OelwjEttIBB08r3Z5umQL0A03lp2Gd
# Secret key: Q3bcPKvbvlVpzv5BQe3lj7EkWdRhevEp24Oi7TENce6xO0FiXUNQKDa47QTyyKcK

In [2]:
# Global variables for storing price data
timestamps = []
price = {'open':[], 'high':[], 'low':[], 'close':[]}
# Maximum number of messages to receive before closing websocket
max_messages = 5 
data = ''

# Initial dashboard graph
fig = go.Figure()

def time_conv(timestamp):
    return datetime.datetime.fromtimestamp(timestamp/ 1000).strftime('%Y-%m-%d %H:%M:%S.%f')[:-4]

def update_graph():
    fig.add_trace(go.Candlestick())

    # Update the graph
    fig.update_xaxes(range=[min(timestamps), max(timestamps)])
    fig.update_yaxes(range=[min(price['low']), max(price['high'])])
    fig.update_traces(
        x=timestamps,
        open=price['open'],
        high=price['high'],
        low=price['low'],
        close=price['close']
    )
    fig.update_layout(
        xaxis=dict(
        type="date",
        tickformat="%H:%M:%S %Y-%m-%d"  # Customize the date format
        ),
        yaxis=dict(
            tickformat=".2f"
        )
    )
    #return fig

def on_error(ws, error):
    print("WebSocket error:", error)

def on_open(ws):
    print("WebSocket connection opened")

def on_close(ws):
    if ws.sock and ws.sock.connected:
        print("WebSocket connection closed.")
    else:
        print("WebSocket connection is already closed.")



def on_message(ws, message):
    global data, orig
    orig = json.loads(message)

    # Append timestamp and price to the lists
    timestamps.append(time_conv(orig['E']))
    data = orig['k']
    price['open'].append(data['o'])
    price['high'].append(data['h'])
    price['low'].append(data['l'])
    price['close'].append(data['c'])

    # Update dashboard graph with new data
    #update_graph()

    # Terminate the WebSocket connection after reaching max number of messages
    if len(timestamps) >= max_messages:
        if ws.sock and ws.sock.connected:
            ws.close()
        else:
            print("WebSocket connection is already closed.")

# Binance WebSocket API endpoint for BTCUSDT symbol
ws_url = 'wss://testnet.binance.vision/ws/btcusdt@kline_1m'

# Start WebSocket connection
ws = websocket.WebSocketApp(
        url=ws_url,
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close
    )
ws.run_forever()
update_graph()
fig.show()

WebSocket connection opened


In [13]:
count = 0
def on_message(ws, message):
    print(message)
    global count
    count +=1 
    # Terminate the WebSocket connection after reaching max number of messages
    if count >= max_messages:
        if ws.sock and ws.sock.connected:
            ws.close()
        else:
            print("WebSocket connection is already closed.")

# Binance WebSocket API endpoint for BTCUSDT symbol
ws_url = 'wss://testnet.binance.vision/ws/btcusdt@kline_1m'

# Start WebSocket connection
ws = websocket.WebSocketApp(
        url=ws_url,
        on_message=on_message
    )
ws.run_forever()

{"e":"kline","E":1685916602441,"s":"BTCUSDT","k":{"t":1685916600000,"T":1685916659999,"s":"BTCUSDT","i":"1m","f":4328815,"L":4328817,"o":"27250.58000000","c":"27250.58000000","h":"27250.58000000","l":"27250.58000000","v":"0.00843800","n":3,"x":false,"q":"229.94039404","V":"0.00843800","Q":"229.94039404","B":"0"}}
{"e":"kline","E":1685916604521,"s":"BTCUSDT","k":{"t":1685916600000,"T":1685916659999,"s":"BTCUSDT","i":"1m","f":4328815,"L":4328821,"o":"27250.58000000","c":"27250.59000000","h":"27250.59000000","l":"27250.57000000","v":"0.05886900","n":7,"x":false,"q":"1604.21439083","V":"0.05772200","Q":"1572.95798290","B":"0"}}
{"e":"kline","E":1685916607127,"s":"BTCUSDT","k":{"t":1685916600000,"T":1685916659999,"s":"BTCUSDT","i":"1m","f":4328815,"L":4328823,"o":"27250.58000000","c":"27250.59000000","h":"27250.59000000","l":"27250.57000000","v":"0.11564800","n":9,"x":false,"q":"3151.47563044","V":"0.11350100","Q":"3092.96864251","B":"0"}}
{"e":"kline","E":1685916609132,"s":"BTCUSDT","k":{"

False

In [3]:
print("Range:", fig.layout.xaxis.range)
print("Range:", fig.layout.yaxis.range)


Range: ('2023-06-04 14:32:34.48', '2023-06-04 14:32:44.39')
Range: ('27211.27000000', '27216.18000000')


In [4]:
price

{'open': ['27216.18000000',
  '27216.18000000',
  '27216.18000000',
  '27216.18000000',
  '27216.18000000'],
 'high': ['27216.18000000',
  '27216.18000000',
  '27216.18000000',
  '27216.18000000',
  '27216.18000000'],
 'low': ['27211.75000000',
  '27211.75000000',
  '27211.75000000',
  '27211.27000000',
  '27211.27000000'],
 'close': ['27211.83000000',
  '27211.84000000',
  '27211.84000000',
  '27211.27000000',
  '27211.27000000']}

In [6]:
ws.close()

In [7]:
data

{'t': 1685914320000,
 'T': 1685914379999,
 's': 'BTCUSDT',
 'i': '1m',
 'f': 4327081,
 'L': 4327128,
 'o': '27216.18000000',
 'c': '27211.27000000',
 'h': '27216.18000000',
 'l': '27211.27000000',
 'v': '0.91279700',
 'n': 48,
 'x': False,
 'q': '24840.89642342',
 'V': '0.15392800',
 'Q': '4189.08144566',
 'B': '0'}

In [8]:
timestamps

['2023-06-04 14:32:34.48',
 '2023-06-04 14:32:36.86',
 '2023-06-04 14:32:39.63',
 '2023-06-04 14:32:42.25',
 '2023-06-04 14:32:44.39']

In [2]:
print(os.environ)

environ({'COMMAND_MODE': 'unix2003', 'HOME': '/Users/isaiahtp', 'LOGNAME': 'isaiahtp', 'MallocNanoZone': '0', 'OLDPWD': '/', 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined', 'PATH': '/Users/isaiahtp/Desktop/Zeppelin/z-algo/.venv/bin:/Users/paulterrell-perica/opt/anaconda3/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Users/isaiahtp/Library/Python/3.9/lib/python/site-packages:/Users/isaiahtp/Library/Python/3.9/bin:/Applications/VMware Fusion.app/Contents/Public:/Library/TeX/texbin:/usr/local/share/dotnet:~/.dotnet/tools:/Library/Apple/usr/bin', 'PWD': '/', 'SHELL': '/bin/zsh', 'SHLVL': '1', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.0FhPER2ADO/Listeners', 'TMPDIR': '/var/folders/jn/6_8h_cj14rv1f9p8_j_fhxfh0000gn/T/', 'USER': 'isaiahtp', 'VSCODE_AMD_ENTRYPOINT': 'vs/workbench/api/node/extensionHostProcess', 'VSCODE_CODE_CACHE_PATH': '/Users/isaiahtp/Library/Application Support/Code/CachedData/b3e4e68a0bc097f0ae7907b217c1119af9e03435', 'VSCODE_CRASH_REP